In [1]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [2]:
train_path = '../data_to_model/keras_data/model_df_full_narrative.csv'
train_df = pd.read_csv(train_path)
train_df

,cpsc_case_number,narrative,primary_mechanism
0,100145411,67 yof pt fell on the floor playing pickle bal...,"Falls, trips, etc."
1,100417928,63yof playing pickleball in the hot sun and pa...,Heat stroke
2,100417973,66yom strained lower leg playing pickle ball,Other mechanism
3,100417997,60yof pt playing pickle ball and fell sustaine...,"Falls, trips, etc."
4,100454409,70 yom pt injured knee while playing pickle ba...,Other mechanism
...,...,...,...
1986,200101825,68 yof tripped and fell while playing tennis. ...,"Falls, trips, etc."
1987,200102934,76yof was hit in the eye with a tennis ball dx...,Hit with various obj
1988,200110467,61yof was playing tennis when she developed pa...,Undetermined/unknown
1989,200124689,70yof fell while playing tennis dx: strained u...,"Falls, trips, etc."


In [3]:
text_np = np.array(train_df['narrative'])
train_labels_np = np.array(train_df['primary_mechanism'])
train_labels_np_bin = np.where(train_labels_np == 'Multi-sport', 1, 0)
print(train_labels_np[0:30])
print(train_labels_np_bin[0:30])

['Falls, trips, etc.' 'Heat stroke' 'Other mechanism' 'Falls, trips, etc.'
 'Other mechanism' 'Falls, trips, etc.' 'Falls, trips, etc.'
 'Other mechanism' 'Other mechanism' 'Falls, trips, etc.'
 'Hit with various obj' 'Falls, trips, etc.' 'Falls, trips, etc.'
 'cardio/sync' 'Falls, trips, etc.' 'Other mechanism' 'Other mechanism'
 'Other mechanism' 'Other mechanism' 'Falls, trips, etc.'
 'Falls, trips, etc.' 'Hit with various obj' 'Other mechanism'
 'Multi-sport' 'Falls, trips, etc.' 'Other mechanism' 'Falls, trips, etc.'
 'Falls, trips, etc.' 'Falls, trips, etc.' 'Falls, trips, etc.']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


In [ ]:
#l_list = []
#for txt in text_np:
#    l_list.append( len(txt) )
    
#l_list[0:10]

#max(l_list)

In [4]:
maxlen = 207
training_samples = 1491
validation_samples = 500
max_words = 100000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_np)
sequences = tokenizer.texts_to_sequences(text_np)


In [5]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=maxlen)

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', train_labels_np.shape)


Found 1857 unique tokens.
Shape of data tensor: (1991, 207)
Shape of label tensor: (1991,)


In [6]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
train_labels_np_bin = train_labels_np_bin[indices]
x_train = data[:training_samples]
y_train_bin = train_labels_np_bin[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val_bin = train_labels_np_bin[training_samples: training_samples + validation_samples]

In [ ]:
data.shape

In [7]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
model = Sequential()
model.add(Embedding(max_words, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 207, 8)            800000    
_________________________________________________________________
flatten (Flatten)            (None, 1656)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 11599     
Total params: 811,599
Trainable params: 811,599
Non-trainable params: 0
_________________________________________________________________


In [9]:
history = model.fit(x_train, y_train_bin,
epochs=80,
batch_size=32,
validation_data=(x_val, y_val_bin))

Epoch 1/80
47/47 [==============================] - 1s 14ms/step - loss: 0.1950 - accuracy: 0.1254 - val_loss: 0.2151 - val_accuracy: 0.2060
Epoch 2/80
47/47 [==============================] - 0s 10ms/step - loss: 0.1950 - accuracy: 0.1261 - val_loss: 0.2151 - val_accuracy: 0.0900
Epoch 3/80
47/47 [==============================] - 0s 9ms/step - loss: 0.1950 - accuracy: 0.1489 - val_loss: 0.2151 - val_accuracy: 0.1800
Epoch 4/80
47/47 [==============================] - 0s 9ms/step - loss: 0.1950 - accuracy: 0.1623 - val_loss: 0.2151 - val_accuracy: 0.1160
Epoch 5/80
47/47 [==============================] - 0s 9ms/step - loss: 0.1950 - accuracy: 0.1482 - val_loss: 0.2151 - val_accuracy: 0.1440
Epoch 6/80
47/47 [==============================] - 0s 9ms/step - loss: 0.1950 - accuracy: 0.1697 - val_loss: 0.2151 - val_accuracy: 0.1580
Epoch 7/80
47/47 [==============================] - 0s 9ms/step - loss: 0.1950 - accuracy: 0.1516 - val_loss: 0.2151 - val_accuracy: 0.2460
Epoch 8/80
47/47 [

In [10]:
model.save_weights('../Model Objects/keras_out_bin_v1.h5')

In [ ]:
def getModel():
    my_dim = 8
    
    model = Sequential()
    model.add(Embedding(max_words, my_dim, input_length=maxlen))
    model.add(Flatten())
#    model.add(Dense(32, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    
    return model

In [ ]:
# Different configurations to try
epochs_list = [30, 60, 90]
batch_sizes = [16, 32, 64]
#embedding_dim = 8
# Dictionary to store the history of each configuration
history_dict = {}

for epochs in epochs_list:
    for batch_size in batch_sizes:
        # Get a fresh instance of the model for each run
        model = getModel()

        # Compile the model
        model.compile(optimizer='rmsprop',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        # Train the model
        history = model.fit(x_train, y_train_one_hot,
                            epochs=epochs,
                            batch_size=batch_size,
                            validation_data=(x_val, y_val_one_hot),
                            verbose=2)  # Set verbose to 2 for less output

        # Save the history of this configuration
        history_dict[f"epochs_{epochs}_batch_{batch_size}"] = history.history

        # Optionally, you can print out the final validation accuracy for each configuration
        final_val_accuracy = history.history['val_accuracy'][-1]
        print(f"Epochs: {epochs}, Batch size: {batch_size}, Validation accuracy: {final_val_accuracy}")

# Now you can analyze the history_dict to see which configuration performed best


In [ ]:
history_dict